# Fluvial flood hazard workflow

In [ ]:
# Import packages
import subprocess
from pathlib import Path

from hydroflows import Workflow, WorkflowConfig
from hydroflows.log import setuplog
from hydroflows.methods import discharge, sfincs, wflow
from hydroflows.utils.example_data import fetch_data

logger = setuplog(level="INFO")

In [ ]:
# Fetch the global build data
cache_dir = fetch_data(data="global-data")

In [ ]:
# Define case name and root directory
name = "fluvial_hazard"
pwd = Path().resolve()  # Get the current file location
case_root = Path(pwd, "cases", name)  # output directory

In [ ]:
# Setup the configuration
config = WorkflowConfig(
    # general settings
    region=Path(pwd, "data/build/region.geojson"),
    catalog_path=Path(cache_dir, "data_catalog.yml"),
    plot_fig=True,
    start_date="2014-01-01",
    end_date="2021-12-31",
    # sfincs settings
    hydromt_sfincs_config=Path(pwd, "hydromt_config/sfincs_config.yml"),
    sfincs_exe=Path(pwd, "bin/sfincs_v2.1.1/sfincs.exe"),
    depth_min=0.05,  # minimum depth for inundation map
    # wflow settings
    hydromt_wflow_config=Path(pwd, "hydromt_config/wflow_config.yml"),
    wflow_exe=Path(pwd, "bin/wflow_v0.8.1/bin/wflow_cli.exe"),
    # design event settings
    rps=[2, 5, 10],
)


In [ ]:
# create and empty workflow
wf = Workflow(name=name, config=config, root=case_root)

In [ ]:
# Build SFINCS model
sfincs_build = sfincs.SfincsBuild(
    region=wf.get_ref("$config.region"),
    config=wf.get_ref("$config.hydromt_sfincs_config"),
    sfincs_root="models/sfincs",
    catalog_path=wf.get_ref("$config.catalog_path"),
    plot_fig=wf.get_ref("$config.plot_fig"),
)
wf.add_rule(sfincs_build, rule_id="sfincs_build")

In [ ]:
# Build wflow model
# additional "gauges" are created at sfincs discharge source points
wflow_build = wflow.WflowBuild(
    region=sfincs_build.output.sfincs_region,
    config=wf.get_ref("$config.hydromt_wflow_config"),
    wflow_root="models/wflow",
    catalog_path=wf.get_ref("$config.catalog_path"),
    gauges=Path(sfincs_build.params.sfincs_root, "gis", "src.geojson"),
    plot_fig=wf.get_ref("$config.plot_fig"),
)
wf.add_rule(wflow_build, rule_id="wflow_build")


In [ ]:
# Update forcing & run wflow model
wflow_update = wflow.WflowUpdateForcing(
    wflow_toml=wflow_build.output.wflow_toml,
    catalog_path=wf.get_ref("$config.catalog_path"),
    start_time=wf.get_ref("$config.start_date"),
    end_time=wf.get_ref("$config.end_date"),
)
wf.add_rule(wflow_update, rule_id="wflow_update")

In [ ]:
# Run the wflow model
wflow_run = wflow.WflowRun(
    wflow_toml=wflow_update.output.wflow_out_toml,
    wflow_bin=wf.get_ref("$config.wflow_exe"),
)
wf.add_rule(wflow_run, rule_id="wflow_run")

In [ ]:
# Derive fluvial design events
fluvial_events = discharge.FluvialDesignEvents(
    discharge_nc=wflow_run.output.wflow_output_timeseries,
    rps=wf.get_ref("$config.rps"),
    wildcard="fluvial_events",
    event_root="input/events",
    index_dim="Q_gauges_bounds",
)
wf.add_rule(fluvial_events, rule_id="fluvial_events")

In [ ]:
# Prepare sfincs models per event, run & postprocess
sfincs_update = sfincs.SfincsUpdateForcing(
    sfincs_inp=sfincs_build.output.sfincs_inp,
    event_yaml=fluvial_events.output.event_yaml,
)
wf.add_rule(sfincs_update, rule_id="sfincs_update")

In [ ]:
# Run the Sfincs model(s)
sfincs_run = sfincs.SfincsRun(
    sfincs_inp=sfincs_update.output.sfincs_out_inp,
    sfincs_exe=wf.get_ref("$config.sfincs_exe"),
)
wf.add_rule(sfincs_run, rule_id="sfincs_run")

In [ ]:
# Postprocess the sfincs output
sfincs_post = sfincs.SfincsDownscale(
    sfincs_map=sfincs_run.output.sfincs_map,
    sfincs_subgrid_dep=sfincs_build.output.sfincs_subgrid_dep,
    depth_min=wf.get_ref("$config.depth_min"),
    output_root="output/hazard",
    event_name="{fluvial_events}",
)
wf.add_rule(sfincs_post, rule_id="sfincs_post")

In [ ]:
# Test the workflow
wf.dryrun()

In [ ]:
# Write the workflow to a Snakefile and snakefile.config.yml
wf.to_snakemake()

# show the files in the case directory
print(f"{wf.root.relative_to(pwd)}:")
for f in wf.root.iterdir():
    print(f"- {f.name}")

In [ ]:
from IPython.display import SVG

# (test) run the workflow with snakemake and visualize the directed acyclic graph
# make sure to have snakemake installed in your environment
subprocess.run('snakemake --dag | dot -Tsvg > dag.svg', cwd=wf.root, shell=True).check_returncode()

# show the dag
SVG(Path(wf.root, "dag.svg").as_posix())

In [ ]:
# uncomment to run the workflow
# subprocess.run(["snakemake", "-c", "1"], cwd=wf.root)